# About this Notebook

This notebook shall be used for inference. For a given list of DSIs and possibly the according adaptive threshold filters, a trained model is loaded and then applied receive depthmap estimations.

It relies on the classes and functions defined in *Classes_and_Functions.ipynb*

# Dependencies

In [ ]:
# Standard library imports
import random
import os
import gc
import re
import time

# Third-party library imports
import numpy as np
import matplotlib.pyplot as plt
import cv2  # OpenCV for adaptive filtering
import psutil  # For system resource management
from scipy.ndimage import convolve  # To convolve filtering masks

# PyTorch specific imports
import torch
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset, Subset
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# Notebooks
import import_ipynb
from Classes_and_Functions import *

# Load Model

First, we load our trained models. For a runtime analysis, only load a single model: set <b>num_models = 1</b>. To leverage ensemble learning, set <b>num_models > 1</b> and load the parameters for each individual model.

In [ ]:
num_models = 1 #  How many models. Set to 1 for runtime analysis.
multi_pixel = False #  Single or multi-pixel version of the network
sub_frame_radius_h = 3 #  Radius of the Sub-DSIs
sub_frame_radius_w = 3

In [ ]:
# Initialize models
models = [PixelwiseConvGRU(sub_frame_radius_h, sub_frame_radius_w, multi_pixel=multi_pixel) for _ in range(num_models)]
# Send to cuda
if torch.cuda.is_available():
    for model in models:
        model.cuda()
# Print architecture
print(models[0])

In [ ]:
# Set path to load models from directory
model_paths = [None] * num_models
# Give names of model files
model_files = ["final/mvsec_stereo/flying1_stereo_model_even", "final/mvsec_stereo/flying1_stereo_model_odd"] #  ["test/example_model_epoch_1"] * num_models
# Do not forget ".pth"
for idx, model_file in enumerate(model_files):
    if not model_file.endswith(".pth"):
        model_files[idx] += ".pth"

In [ ]:
# Load models parameters
for idx, model in enumerate(models):
    model.load_parameters(model_files[idx], device=device, model_path=model_paths[idx], optimizer=None)

In [ ]:
# Use ensemble learning to create averaged model
model = AveragedNetwork(models)

# Data

We want to estimate depth for a given set of DSIs and adaptive gaussian threshold filters. If DSIs are not given yet, load them. If Threshold filters are not given yet, compute them.

In [ ]:
dsi_list = None #  Give here
threshold_mask_list = None #  Give here

In [ ]:
# Load DSIs from a directory
dsi_directory = "/home/diego/Stereo Depth Estimation/data/mvsec/indoor_flying1/dsi/" #  "example_dsi_directory"
start_idx, end_idx = 0, None
if dsi_list is None or dsi_list == []:
    dsi_list = load_dsi_list(dsi_directory, start_idx, end_idx)

In [ ]:
# Original filter parameters for MVSEC indoor flying and DSEC zurich city 04a
"""
Mvsec indoor flying 1,2,3:
    Original: filter_size = 5 | adaptive_threshold_c = -14
    Denser: filter_size = 9 | adaptive_threshold_c = -10
    max_confidence =  [57.7, 78, 78.8]
DSEC zurich city 04a:
    filter_size = 5 | adaptive_threshold_c = -4 | max_confidence = 468
"""

# Choose filter parameters
filter_size = 9 #  (int): Determines the size of the neighbourhood area when applying the adaptive threshold filter
adaptive_threshold_c = -14 #  (int): Constant that is subtracted from the mean of the neighbourhood pixels when apply the adaptive threshold filter.
max_confidence = 57.7 #  (int): The maximum relevant ray count in the DSI sequence.

In [ ]:
# Create threshold masks
if threshold_mask_list is None or threshold_mask_list == []:
    threshold_mask_list = [get_threshold_mask(dsi, filter_size, adaptive_threshold_c, max_confidence) for dsi in dsi_list]

# Inference

The estimated depthmaps are generated by creating an instance of the *Estimated_Depthmaps* class.

They are stored under <b>self.estimated_depths</b>.

In [ ]:
st = time.time()
inference = Estimated_Depthmaps(model,
                                dsi_list,
                                threshold_mask_list,
                                # Input creation (Sub-DSIs)
                                sub_frame_radius_h=sub_frame_radius_h,
                                sub_frame_radius_w=sub_frame_radius_w,
                                batch_size=1024
                               )
ct = time.time()
print("Inference Time [sec]:", round(ct-st,2), "\n")
# print(inference.estimated_depths)

# Visualize

The method <b>self.create_colored_depth_estimations()</b> applies the 'jet' colormap to them.

The colored depthmaps are then stored under <b>self.colored_depth_estimations</b>.

In [ ]:
# Color depthmaps
inference.create_colored_depth_estimations()

In [ ]:
# Running the following cell will plot the colored depthmap estimations
figsize=(10,6)
for depthmap in inference.colored_depth_estimations:
    plt.figure(figsize=figsize)
    plt.imshow(depthmap)
    plt.show()